###### Content under Creative Commons Attribution license CC-BY 4.0, code under BSD 3-Clause License © 2017 L.A. Barba, N.C. Clementi

# Lesson 2 title?

Welcome to Lesson 2 of the course module "Fly at changing systems," in _Engineering Computations_. The previous lesson, [Catch things in motion](http://go.gwu.edu/engcomp3lesson1), showed you how to compute velocity and acceleration of a moving body whose positions were known. 

Time history of position can be captured on a long-exposure photograph (using a strobe light), or on video. But digitizing the positions from images can be a bit tedious, and error-prone. Luckily, we found online a data set from a fancy motion-capture experiment of a falling ball, with high resolution [1]. You computed acceleration and found that it was not only smaller than the theoretical value of $9.8\rm{m/s}^2$, but it _decreased_ over time. The effect is due to air resistance and is what leads to objects reaching a _terminal velocity_ in freefall.

In general, not only is [motion capture](https://en.wikipedia.org/wiki/Motion_capture) (a.k.a., _mo-cap_) expensive, but it's inappropriate for many physical scenarios. Take a roller-coaster ride, for example: during design of the ride, it's more likely that the engineers will use an _accelerometer_. It really is the acceleration that makes a roller-coaster ride exciting, and they only rarely go faster than highway speeds (say, 60 mph) [2].
How would an engineer analyze data captured with an accelerometer?

## A roller-coaster ride

Prof. Anders Malthe-Sorenssen has a file with accelerometer data for a roller-coaster ride called "The Rocket" (we don't know if it's real or made up!). He has kindly given permission to use his data. So let's load it and have a look. We'll first need our favorite numerical Python libraries, of course.

In [ ]:
import numpy
from matplotlib import pyplot

In [ ]:
%matplotlib inline

#Import rcParams to set font styles
from matplotlib import rcParams

#Set font style and size 
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 14

If you don't have the data file in the location we assume below, you can get it by adding a code cell and executing this code in it, then commenting or deleting the `filename` assignment before the call to `numpy.loadtxt()`.

```Python
from urllib.request import urlretrieve
URL = 'http://go.gwu.edu/engcomp3data2?accessType=DOWNLOAD'
urlretrieve(URL, 'therocket.txt')
```

In [ ]:
filename = '../../data/therocket.txt'
t, a = numpy.loadtxt(filename, usecols=[0,1], unpack=True)

We'll take a peek at the data by printint the first five pairs of $(t, a)$ values, then plot the whole set below. Time is given in units of seconds, while acceleratino is in $\rm{m/s}^2$.

In [ ]:
for i in range(0,5): print(t[i],a[i])

In [ ]:
fig = pyplot.figure(figsize=(8, 3))

pyplot.plot(t, a, color='#004065', linestyle='-', linewidth=1) 
pyplot.title('Acceleration of roller-coster ride data, from [1]. \n')
pyplot.xlabel('Time [s]')
pyplot.ylabel('[m/s$^2$]');

Our challenge now is to find the motion description—the position $x(t)$—from the acceleration data. In the previous lesson, we did the opposite: with position data, get the velocity and acceleration, using _numerical derivatives_:

\begin{equation}
v(t_i) = \frac{\rm{d}x}{\rm{d}t} \approx \frac{x(t_i+\Delta t)-x(t_i)}{\Delta t}
\end{equation}

\begin{equation}
a(t_i) = \frac{\rm{d}v}{\rm{d}t} \approx \frac{v(t_i+\Delta t)-v(t_i)}{\Delta t}
\end{equation}

Since this time we're dealing with horizontal acceleration, we swapped the position variable from $y$ to $x$ in the equation for velocity, above. 

The key to our problem is realizing that if we have the initial velocity, we can use the acceeration data to find the velocity after a short interval of time. And if we have the initial position, we can use the known velocity to find the new position after a short interval of time. Let's rearrange the equation for acceleration above, by solving for the velocity at $t_i + \Delta t$:

\begin{equation}
 v(t_i+\Delta t)   \approx  v(t_i) + a(t_i) \Delta t
\end{equation}

We need to know the velocity and acceleration at some initial time, $t_0$, and then we can compute the velocity $v(t_i + \Delta t)$. For the roller-coaster ride, it's natural to assume that the initial velocity is zero, and the initial position is zero with respect to a convenient reference system. We're actually ready to solve this!

Let's save the time increment for our data set in a variable named `dt`, and compute the number of time increments in the data. Then, we'll initialize new arrays of velocity and position to all-zero values, with the intention of updating these to the computed values.

In [ ]:
dt = t[1]-t[0]
dt

In [ ]:
N = len(t)
N

In [ ]:
v = numpy.zeros(N)
x = numpy.zeros(N)

In the code cell below, we use a `for` statement to step through the sequence of acceleration values, each time computing the velocity and position at the subsequent time instant. We are applying the equation for $v(t_i + \Delta t)$ above, and a similar equation for position:

\begin{equation}
 x(t_i+\Delta t)   \approx  x(t_i) + v(t_i) \Delta t
\end{equation}

In [ ]:
for i in range(N-1):
    v[i+1] = v[i] + a[i]*dt
    x[i+1] = x[i] + v[i]*dt

And there you have it. You have computed the velocity and position over time from the acceleration data. 

In [ ]:
fig = pyplot.figure(figsize=(10,6))

pyplot.subplot(211)
pyplot.plot(t,  v, color='#0096d6', linestyle='-', linewidth=1) 
pyplot.title('Velocity and position of roller-coster ride (data from [1]). \n')
pyplot.ylabel('$v$ [m/s] ')

pyplot.subplot(212)
pyplot.plot(t,  x, color='#008367', linestyle='-', linewidth=1) 
pyplot.xlabel('Time [s]')
pyplot.ylabel('$x$ [m]');

## Euler's method


## References

1. _Elementary Mechanics Using Python_ (2015), Anders Malthe-Sorenssen, Undergraduate Lecture Notes in Physics, Springer. Data at http://folk.uio.no/malthe/mechbook/

2. _The Physics Hyptertextbook_ (n/a), Glenn Elert, [Acceleration](https://physics.info/acceleration/)

In [ ]:
# Execute this cell to load the notebook's style sheet, then ignore it
from IPython.core.display import HTML
css_file = '../../style/custom.css'
HTML(open(css_file, "r").read())